# Eat Safe, Love

## Notebook Set Up

In [831]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [832]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [833]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [834]:
# review the collections in our database
collections = db.list_collection_names()
collections

['establishments']

In [835]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [836]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Create a list to store the documents from the query
document_list = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
count
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'Latitude': 0.0,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'Longitude': 0.0,
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6515cdd70f27b03b01487af4'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta':

In [837]:
# Convert the result to a Pandas DataFrame
#initial approach was to do df = pd.DataFrame((establishments.find(query))) but that didn't work
#had a problem - when transforming a query to a dataframe mongoDB flattens the document
#and removes fields that do not exist in all documents, which removed RatingValue and other cols from the df
#fixed using code below
document_list = list(establishments.find(query))

# get unique fields for all docs
all_fields = set()
for document in document_list:
    all_fields.update(document.keys())

# use None for missing fields
for document in document_list:
    for field in all_fields:
        if field not in document:
            document[field] = None

#flatten/ normalize json when creating df
df = pd.json_normalize(document_list)

# Display the number of rows in the DataFrame
len(df)

# Display the first 10 rows of the DataFrame
df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,6515cdd70f27b03b01487af4,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,0.27694,50.769705,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,6515cdd70f27b03b01487e73,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,1.278721,51.857536,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,6515cdd70f27b03b01488184,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,1.15927,51.789429,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,6515cdd70f27b03b01488378,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,0.101446,50.770724,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,6515cdd70f27b03b0148837f,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,0.10202,50.770885,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,6515cdd70f27b03b01488d1e,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,0.736349,51.541448,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,6515cdd70f27b03b01488f3d,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,0.7121671,51.5350065,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,6515cdd70f27b03b0148a468,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,0.602364,51.591515,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,6515cdd70f27b03b0148a8b0,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,0.146449998021126,51.1859474182129,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,6515cdd70f27b03b0148b0d6,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,0.462307,51.57005,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [838]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4
london_regex = "^London"
query = {
    'LocalAuthorityName': {"$regex": london_regex},
    'RatingValue': {"$gte": 0}
}
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
# Display the first document in the results using pprint
first = establishments.find_one(query)

In [839]:
# Convert the result to a Pandas DataFrame
londonquery = establishments.find(query)
londondf = pd.DataFrame(londonquery)
# Display the number of rows in the DataFrame
len(londondf)
# Display the first 10 rows of the DataFrame
londondf.head(10)

""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [840]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
#longitude":"0.08384000",
#"latitude":"51.49014200"

degrees = 0.01
#used lat and lon given for other notebook
latitude = 51.49014200
longitude = 0.08384000

query = {
    "geocode.latitude": {
        "$gte": latitude - degrees,
        "$lte": latitude + degrees
    },
    "geocode.longitude": {
        "$gte": longitude - degrees,
        "$lte": longitude + degrees
    },
    "RatingValue": 5
}

sort = [("scores.Hygiene", 1)]

#top 5 closest
top5 = establishments.find(query).sort(sort).limit(5)

# Print the results
top5

In [844]:
# Convert result to Pandas DataFrame
top5df = pd.DataFrame(top5)
top5df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [842]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [843]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
